In [0]:
""" The codes for the projects were adapted from the https://github.com/siddk/npi 
This cell is for the data generation"""

from generate_data import create_trace


def create_data(train_samples_number = 30, test_samples_number = 16):
    create_trace('train', train_samples_number)
    create_trace('test', test_samples_number)

if __name__ == "__main__":

    print('Creating data ...... ')
    create_data()


In [0]:
"""This cell is for training"""
from npi import NPI
from f_encoder import F_Encoder
from config import CONFIG, ScratchPad
import pickle
import tensorflow as tf

INCREMENT = 1
DATA_PATH = "train.pik"

def train_model(epochs):
    """
    Instantiates an Addition Core, NPI, then loads and fits model to data.

    :param epochs: Number of epochs to train for.
    """
    # Load Data
    with open(DATA_PATH, 'rb') as f:
        data = pickle.load(f)

    # Initialize Addition Core
    print('Initializing Core!')
    core = F_Encoder()

    # Initialize NPI Model
    print('Initializing NPI Model!')
    npi = NPI(core, CONFIG)

    # Initialize TF Saver
    saver = tf.train.Saver()

    # Initialize TF Session
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())

    # Start Training
    training_error = {}

    for ep in range(1, epochs + 1):
        print(ep)
        training_error[ep] = []
        for i in range(len(data)):
            # Reset NPI States
            npi.reset_state()

            # Setup Environment
            c_find, steps = data[i]
            scratch = ScratchPad(c_find)
            x, y = steps[:-1], steps[1:]
           # Run through steps, and fit!
            step_def_loss, term_acc, prog_acc, = 0.0, 0.0, 0.0

            for j in range(len(x)):
                (prog_name, prog_in_id), arg, term = x[j]
                (_, prog_out_id), arg_out, term_out = y[j]

                if prog_in_id == 1:
                    scratch.increment_ptr()
                # Get Environment, Argument Vectors
                env_in = [scratch.get_env()]
                # arg_in, arg_out = [get_args(arg, arg_in=True)], get_args(arg_out, arg_in=False)
                prog_in, prog_out = [[prog_in_id]], [prog_out_id]
                term_out = [1] if term_out else [0]

                # Fit!
                loss, t_acc, p_acc, _ = sess.run(
                        [npi.default_loss, npi.t_metric, npi.p_metric, npi.default_train_op],
                        feed_dict={npi.env_in: env_in,  npi.prg_in: prog_in,
                                   npi.y_prog: prog_out, npi.y_term: term_out})
                step_def_loss += loss
                term_acc += t_acc
                prog_acc += p_acc

            training_error[ep].append((i, step_def_loss / len(x), term_acc / len(x), prog_acc / len(x)))
    # Save Model
    print("Epoch {} Step {} Default Step Loss {},Term: {}, Prog: {}".format(ep, i, step_def_loss / len(x), term_acc / len(x), prog_acc / len(x)))

    saver.save(sess, 'model/model.ckpt')
    return training_error

if __name__ == "__main__":
    training_error = train_model(epochs=30)


In [0]:
"""Plotting"""
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(7,5)})

# plot first epoch
err1 = training_error[1]
err5 = training_error[5]
err10 = training_error[30]

step_def_loss1 = [i[1] for i in err1]
prog_acc1 = [i[2] for i in err1]
term_acc1 = [i[3] for i in err1]

step_def_loss5 = [i[1] for i in err5]
prog_acc5 = [i[2] for i in err5]
term_acc5 = [i[3] for i in err5]

step_def_loss10 = [i[1] for i in err10]
prog_acc10 = [i[2] for i in err10]
term_acc10 = [i[3] for i in err10]

data = [i[0] for i in err1]
epoch_labels = {0: 1, 1: 5, 2:30}

y = [step_def_loss1, step_def_loss5, step_def_loss10]
plt.xlabel("Data trace num")
plt.ylabel("Step default loss")
for i in range(len(y)):
    plt.plot(data,y[i],label = 'epoch %s'% epoch_labels[i])
plt.legend()
plt.show()

In [0]:
"""Plotting"""
y = [prog_acc1, prog_acc5, prog_acc10]
plt.xlabel("Data trace num")
plt.ylabel("Program accuracy")
for i in range(len(y)):
    plt.plot(data,y[i],label = 'epoch %s'% epoch_labels[i])
plt.legend()
plt.show()

In [0]:
"""Plotting"""
y = [term_acc1, term_acc5, term_acc10]
plt.xlabel("Data trace num")
plt.ylabel("Termination accuracy")
for i in range(len(y)):
    plt.plot(data,y[i],label = 'epoch %s'%epoch_labels[i])
plt.legend()
plt.show()

In [0]:
""" This cell is for testing """
from npi import NPI
from f_encoder import F_Encoder
from config import CONFIG, PROGRAM_SET, ScratchPad, COUNTRY_REGION, NUM_CODE, COUNTRY_REGION_CODE
import numpy as np
import pickle
import tensorflow as tf

CKPT_PATH = "model/model.ckpt"
TEST_PATH = "test.pik"

def evaluate():

    with tf.Session() as sess:
        # Load Data
        with open(TEST_PATH, 'rb') as f:
            data = pickle.load(f)

        # Initialize Core
        core = F_Encoder()

        # Initialize NPI Model
        npi = NPI(core, CONFIG)

        # Restore from Checkpoint
        saver = tf.train.Saver()
        saver.restore(sess, CKPT_PATH)

        # Run REPL
        repl(sess, npi, data)


def repl(session, npi, data):
    while True:
        inpt = input('Hit Enter for Random country: ')

        if inpt == "":
            x = data[np.random.randint(len(data))][0]

        else: 
          break
          
        print('To find: ', COUNTRY_REGION_CODE[x])
        # Reset NPI States
        print("")
        npi.reset_state()

        # Setup Environment
        scratch = ScratchPad(x)
        prog_name, prog_id, arg, term = 'FIND', 0, [], False
        print('--------  Environment ----------' )
        scratch.pretty_print()
        print('-----------------------------------')
        cont = 'c'
        while cont == 'c' or cont == 'C':
            a_str = "[]"

            print('Step: %s, Arguments: %s, Terminate: %s' % (prog_name, a_str, str(term)))
            
            if prog_id == 1:
                scratch.increment_ptr()

            
            env_in, prog_in = [scratch.get_env()],  [[prog_id]]

            t, n_p = session.run([npi.terminate, npi.program_distribution],
                                         feed_dict={npi.env_in: env_in,
                                                    npi.prg_in: prog_in})
            if np.argmax(t) == 1:
                print('Step: %s, Arguments: %s, Terminate: %s' % (prog_name, a_str, str(True)))

                scratch.pretty_print()

                true_ans = None
                for i in COUNTRY_REGION:
                  if i[0] == x:
                      true_ans = COUNTRY_REGION_CODE[i[1]]
                      break
                if(scratch.ptr<scratch.rows-1):
                    output = COUNTRY_REGION_CODE[scratch[scratch.ptr][1]]
                    scratch[scratch.rows-1][0] = NUM_CODE[output]
                else:
                    output = 'Not found'
                
                print('Pointer at ', scratch.ptr)
                print('-------- KB Environment ----------' )
                scratch.pretty_print()
                print('-'*100)
                print("Model Output: ", output)
                print("Correct Out : " , true_ans)
                print("Correct!" if output == true_ans else "Incorrect!")

            else:
                prog_id = np.argmax(n_p)
                prog_name = PROGRAM_SET[prog_id][0]
                term = False

            cont = input('Continue? enter "c"')

if __name__ == "__main__":
    evaluate()
